In [2]:
#Bring in excel file
import pandas as pd
import numpy as np
import matplotlib

#Parse into dataframe
xl = pd.ExcelFile('Capstone_Project_1.xlsx')

#Skip blank rows, parse first first sheet
Wells_DF = xl.parse("CA DOGGR Wells",skiprows=3, index_col=None)

#Filter on OG
Wells_DF= Wells_DF[Wells_DF['Pool WellTypes'].str.contains('OG', na= False)]

#Split on comma
Wells_DF['Field Name'] = Wells_DF['Field Name'].str.split(',').str[0]

#Convert Spud Date from object to date
Wells_DF['SPUD Date'].convert_objects(convert_dates=True)


/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:19: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


0        01/01/1976
1        01/01/1992
2        01/01/2002
3        01/01/2008
4        01/01/2009
5        01/01/2010
6        01/01/2011
7        01/01/2011
8        01/01/2011
9        01/01/2011
10       01/01/2012
11       01/01/2012
12       01/01/2012
13       01/01/2012
14       01/01/2013
15       01/01/2013
16       01/01/2013
17       01/01/2015
18       01/01/2016
19       01/02/1988
20       01/02/1999
21       01/02/2001
24       01/02/2009
25       01/02/2009
27       01/02/2009
28       01/02/2009
29       01/02/2009
30       01/02/2009
31       01/02/2009
35       01/02/2009
            ...    
79224           NaN
79225           NaN
79226           NaN
79227           NaN
79228           NaN
79229           NaN
79230           NaN
79231           NaN
79232           NaN
79233           NaN
79234           NaN
79235           NaN
79236           NaN
79237           NaN
79238           NaN
79239           NaN
79240           NaN
79241           NaN
79242           NaN


In [3]:
import scipy.stats as stats
import statsmodels.api as sm
import numpy as np
import pandas as pd

a = Wells_DF['Longitude']
b = Wells_DF['Latitude']

a_mean = stats.trim_mean(a, 0.1)
b_mean = stats.trim_mean(b, 0.1)

a_skew = stats.skew(a, axis=0, bias=True, nan_policy='propagate')
#more weight in right tail of distribution

b_skew = stats.skew(b, axis=0, bias=True, nan_policy='propagate')
#more weight in right tail of distribution

#going to test whether wells drilled after certain year have similar location or if it has shifted. Use alpha of 0.05.
#p-value is less than alpha, so can reject null for Long, but cannot for Lat. Kruskal is less powerful method for ANOVA

s= '1/1/2014'
a_year_2014 = Wells_DF[Wells_DF['SPUD Date'] >= s]['Longitude']
a_year = Wells_DF[Wells_DF['SPUD Date'] < s]['Longitude']
b_year_2014 = Wells_DF[Wells_DF['SPUD Date'] >= s]['Latitude']
b_year = Wells_DF[Wells_DF['SPUD Date'] < s]['Latitude']

a_k = stats.kruskal(a_year_2014, a_year)
b_k = stats.kruskal(b_year_2014, b_year)

a_skew
b_skew
a_k
b_k

KruskalResult(statistic=8.0855566482182741, pvalue=0.0044619423846561646)

In [46]:
Wells_DF['SPUD Date'] = pd.to_datetime(Wells_DF['SPUD Date'])
Wells_DF['Spud Year'] = Wells_DF['SPUD Date'].apply(lambda x: x.year)


#Computer well count by year for data
Wells_DF['Well Count'] = Wells_DF.groupby(['Spud Year'])['Well #'].count()

#Parse into dataframe
xl = pd.ExcelFile('Capstone_Project_1.xlsx')

#Bring in Oil Price and join two dataframes on Year
Oil_DF = xl.parse("Crude Oil Price",skiprows=2, index_col=None)
Oil_DF['Year'] = Oil_DF['Date'].apply(lambda x: x.year)
Oil_DF.columns = ['Date', 'Oil Price', 'Year']


Oil_DF['Oil Price'].groupby([Oil_DF['Year']]).mean()


#joining table
#Oil_DF = Oil_DF[['Year','Average Annual Oil Price']]
#Small_Well_DF = Wells_DF[['Spud Year','Well Count']]
#Oil_DF.join(Small_Well_DF, lsuffix='Year', rsuffix='Spud Year')


Year
1986    15.036667
1987    19.171667
1988    15.982500
1989    19.640833
1990    24.467500
1991    21.502500
1992    20.563333
1993    18.450833
1994    17.191667
1995    18.439167
1996    22.113333
1997    20.610000
1998    14.446667
1999    19.260833
2000    30.300833
2001    25.947500
2002    26.115000
2003    31.120833
2004    41.443333
2005    56.492500
2006    66.018333
2007    72.318333
2008    99.571667
2009    61.654167
2010    79.395000
2011    94.874167
2012    94.110833
2013    97.905833
2014    93.258333
2015    48.688333
2016    43.144167
2017    50.968000
Name: Oil Price, dtype: float64